This model is using pinecone as a database
For more information: https://docs.pinecone.io/docs/langchain#5-use-pinecone-and-langchain-for-rag

In [10]:
import os
import openai
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationChain
from langchain.prompts.prompt import PromptTemplate

from langchain.schema import (AIMessage,
                              HumanMessage,
                              SystemMessage,)

from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

openai.api_type = "azure"


# For University Credential

openai.api_base = os.getenv("OPENAI_ENDPOINT") 
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_version = "2023-05-15"
llm_model = 'gpt-35-turbo-rfmanrique'
llm_embedding = 'text-embedding-ada-002-rfmanrique'

print(openai.api_base, openai.api_key, openai.api_version, openai.api_type, llm_model)

https://invuniandesai.openai.azure.com/ cf0bd49030ed4aa6a6509be1cd9d604b 2023-05-15 azure gpt-35-turbo-rfmanrique


In [3]:
# Is important that the history from the memory enters as an input (and only the history!)
template="""
You are a customer service AI agent, and you need to identify the animic state of the client.
Query = {query}
"""

prompt = PromptTemplate(
    input_variables=["query"],
    template=template)

In [4]:
llm_chat = ChatOpenAI(temperature = 0, engine= llm_model) # Temperature = 0 means less randomness
summary_window_memory = ConversationSummaryBufferMemory(llm=llm_chat,
                                                        max_token_limit = 40)
conversation = ConversationChain(llm = llm_chat,
                                 verbose = True,
                                 memory = summary_window_memory)

In [5]:
response = conversation.predict(input="Hi, I'm Julian, a client of your company")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, I'm Julian, a client of your company
AI:

> Finished chain.


In [6]:
# 1. Split text into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 10,
    chunk_overlap = 0,) # read about chunk optimal overlap

texts = text_splitter.create_documents([response])
# texts[0].page_content # to retrieve the page content


In [7]:
'''
Instead of using OpenAI embeddings, I use Hughing Face Hub Embeddings
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model=llm_embedding)
'''
from langchain.embeddings import HuggingFaceHubEmbeddings
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')
model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceHubEmbeddings(repo_id=model_name)

c:\Anaconda3\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)
You're using a different task than the one specified in the repository. Be sure to know what you're doing :)


In [8]:
query_result = embeddings.embed_query(texts[0].page_content) # Review HuggingFace alternatives

In [17]:
from langchain_pinecone import Pinecone
index_name = 'thesis-model-1'
search = Pinecone.from_documents(texts, embeddings, index_name=index_name)

In [19]:
query = "Assist"
result = search.similarity_search(query)
result

[Document(page_content='assist you'),
 Document(page_content='assistant'),
 Document(page_content='help you'),
 Document(page_content='with any')]